## Import Packages

In [5]:
#doing all the relevant imports
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np## Import Packages
import cv2 #bringing in OpenCV libraries

import math

from moviepy.editor import VideoFileClip
from IPython.display import HTML

import os

from utils import color_selection,grayscale,gaussian_blur,canny,region_of_interest,hough_lines,draw_hough_lines_extrapolate,weighted_img


## Read in an Image

In [15]:
# Read in the image
image_original = mpimg.imread('test_images/solidWhiteCurve.jpg')
# Note: always make a copy rather than simply using "="
image_original2 = np.copy(image_original)
image_original3 = np.copy(image_original)

image_copy = np.copy(image_original)

In [7]:
# Grayscale the image
image_gray = grayscale(image_copy)
mpimg.imsave('result_images/imageGreyScale.png', image_gray,cmap='gray')


**Image converted to gray scale.**

---

 <img src="https://raw.githubusercontent.com/dannofield/Self-driving-car/master/result_images/imageGreyScale.png" width="380" alt="Gray Scaled">
 <p></p> 
 <p style="text-align: center;"> Grey scaled image output </p> 
 <p></p> 


In [8]:
# Define a kernel size and apply a Gaussian Noise kernel to smooth the image
kernel_size = 3
image_blur_gray = gaussian_blur(image_gray, kernel_size)

# Define our parameters for Canny and apply the Canny transform
low_threshold = 50 #35
high_threshold = 200 #70
image_edges_canny = canny(image_blur_gray, low_threshold, high_threshold)
mpimg.imsave('result_images/imageCanny.png', image_edges_canny,cmap='Greys_r')


**After converting the image to gray scaled we need to remove noise from it. Noise can be removed by blurring it, and then the Canny Edge function was applied**

---

<figure>
 <img src="result_images/imageCanny.png" width="380" alt="Canny Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Canny image output </p> 
 </figcaption>
</figure>
 <p></p> 


In [14]:
# Next we'll create a masked edges image using cv2.fillPoly()
# We are defining a four sided polygon to mask
# Since we have video images of 960x540 and 1280x720 we are applying differente
#shapes depending on the size of the image
imshape = image_original.shape

if(imshape[1] == 960): 	#small image uses this mask
	vertices = [(150,imshape[0]),(480, 300), (490, 300), (imshape[1]-30,imshape[0])]
else:					#large image uses this mask
	vertices = [(int(imshape[1]*6/32),660),(620, 430), (710, 430), (imshape[1]-150,650)]

mask_polygon = np.array([vertices], dtype=np.int32)

#This is just to show the zone we are masking
cv2.line(image_original2, vertices[0],vertices[1], color=[0, 255, 0], thickness=2)
cv2.line(image_original2, vertices[1],vertices[2], color=[0, 255, 0], thickness=2)
cv2.line(image_original2, vertices[2],vertices[3], color=[0, 255, 0], thickness=2)


mpimg.imsave('result_images/imageAreaOfInteres.png', image_original2)



**Region of Interest.
The area inside this green triangle is the region of our interest.**

---

<figure>
 <img src="result_images/imageAreaOfInteres.png" width="380" alt="Area of interes" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Creating a mask for the area of interes </p> 
 </figcaption>
</figure>
 <p></p> 


In [16]:
# Define the Hough transform parameters
# Define the Hough transform parameters
rho = 2                 #2,1 distance resolution in pixels of the Hough grid
theta = np.pi/180       # angular resolution in radians of the Hough grid
threshold = 80          #8,15,1 minimum number of votes (intersections in Hough grid cell) 
                        #meaning at least 15 points in image space need to be associated with each line segment
min_line_length = 10    #40,10 minimum number of pixels making up a line
max_line_gap = 13       #20,5 maximum gap in pixels between connectable line segments


# Make a blank the same size as our image to draw on
# Run Hough on edge detected image
#It returns an image with lines drawn on it. It is a blank image (all black) with lines drawn on it
image_hough_lines = hough_lines(image_edges_canny , rho, theta, threshold, min_line_length, max_line_gap)
mpimg.imsave('result_images/imageHoughLinesUnmasked.png', image_hough_lines)


**Applying Hough Transform to an unmasked image**

---

<figure>
 <img src="result_images/imageHoughLinesUnmasked.png" width="380" alt="Hough Lines" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Hough Lines using unmasked Canny's output</p> 
 </figcaption>
</figure>
 <p></p> 


In [11]:
#Now we are masking canny's output with out polygon
image_masked_edges_canny = region_of_interest(image_edges_canny,mask_polygon)
image_hough_lines_masked = hough_lines(image_masked_edges_canny , rho, theta, threshold, min_line_length, max_line_gap)

#This is only necesary if you want the lines on Canny's image
# Create a "color" binary image to combine with line image
image_masked_edges_canny2 = np.dstack((image_masked_edges_canny, image_masked_edges_canny, image_masked_edges_canny)) 

image_combo_result_on_canny = weighted_img(image_hough_lines_masked ,image_masked_edges_canny2, alpha=0.8, beta=1., teta=0.)
mpimg.imsave('result_images/imageHoughLinesPlusCanny.png', image_combo_result_on_canny)

**Drawing the image on canny's output after masking the image and using Hough Transform**

---

<figure>
 <img src="result_images/imageHoughLinesPlusCanny.png" width="380" alt="Hough Lines Plus Canny' output masked" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Hough Lines Plus Canny's output masked</p> 
 </figcaption>
</figure>
 <p></p> 


In [12]:
# Draw the lines on the original image image_original
image_combo_original= weighted_img(image_original,image_hough_lines_masked , alpha=0.8, beta=1., teta=0.)
mpimg.imsave('result_images/imageHoughLinesPlusOriginal.png', image_combo_original)


**Line Detection - Hough Transform over the initial image**

---

<figure>
 <img src="result_images/imageHoughLinesPlusOriginal.png" width="380" alt="Hough Lines masked Plus Original" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Hough Lines masked Plus Original</p> 
 </figcaption>
</figure>
 <p></p>
 
## Improving the draw_lines() function

**After we had the Hough line segments drawn onto the road, we can see that the Hough function returns a set of lines in the same direction. I draw them with random colors so we can see them all.**
<p></p>

<figure>
    <div style="text-align: center;">
 <img src="result_images/imageHoughLinesPlusCannyLeftLines.png" style='margin-top:0;width: 380px; display: inline-block;' alt="Hough Lines masked Left Line"/>  <img src="result_images/imageHoughLinesPlusCannyRightLine.png" style='margin-top:0;width: 380px; display:inline-block;' alt="Hough Lines masked Right Line" />
        </div>
 <figcaption>
 <p><br></p> 
 <p style="text-align: center;clear: both; "> Individual left and right Hough Lines masked</p> 
 </figcaption>
</figure>
 <p></p>
 

**I printed the information of every single line marking the main lines (right and left) on the road. I printed the starting point (x1,y1), the end point (x2,y2) and with these two points we can calculate the slope and the intersection of every single little line.**

<p>
<code>Left Lines
x1:  280  y1:  461  x2:  320  y2:  430  m:  -0.775  b:  678.0
x1:  437  y1:  344  x2:  467  y2:  320  m:  -0.8  b:  693.6
x1:  293  y1:  462  x2:  353  y2:  412  m:  -0.8333333333333334  b:  706.167
x1:  454  y1:  332  x2:  466  y2:  323  m:  -0.75  b:  672.5
x1:  408  y1:  366  x2:  420  y2:  357  m:  -0.75  b:  672.0
x1:  280  y1:  460  x2:  346  y2:  410  m:  -0.7575757575757576  b:  672.1212121

Right Lines
x1:  482  y1:  311  x2:  876  y2:  538  m:  0.5761421319796954  b:  33.29949238578678
x1:  490  y1:  313  x2:  898  y2:  539  m:  0.553921568627451  b:  41.578431372549005
x1:  711  y1:  434  x2:  787  y2:  477  m:  0.5657894736842105  b:  31.7236842105263
x1:  650  y1:  407  x2:  726  y2:  450  m:  0.5657894736842105  b:  39.23684210526318
</code></p>
    
**If we want to identify the full extent of the lane and marking it clearly as in the example video (P1_example.mp4), we can try to average and/or extrapolate the line segments we see on the images to map out the full extent of the lane lines.
I identified each line as part of the left line or the right line of the road by the slope (negative or positive respectively).
Then I created two arrays with these lines and I averaged them by using <code>np.average</code>.
So we can end up with one single line that represents the set of lines returned by the Hough function.**

<p><code>
Average Left Lines
[ -0.77765152 682.3979798 ]
Average Right Lines
[ 0.56541066 36.45961252]
</code></p>

**The new output draws a single, solid line over the left lane line and a single solid line over the right lane line. The lines start from the bottom of the image and extend out to the top of the region of interest (about 3/5th of the image).**

In [18]:
def draw_lines_extrapolate(img, lines, color=[255, 0, 0], thickness=8):
	if lines is not None:
		left_line = []
		right_line = []
		for line in lines:
			for x1,y1,x2,y2 in line:
				m = (y2-y1)/(x2-x1)	#get slope
				b = y1 - m*x1		#get intercept
				if(m < 0. ):
					left_line.append((m,b))
				else:
					right_line.append((m,b))

		#Did we atleast have something?
		if left_line:
			#Get average of all the left lines found
			left_avg_line = np.average(left_line,axis=0)		
			#Calculate its coordinates from m & b
			avgslope,avgintercept = left_avg_line			
            #Do not use faulty data 
			if not (avgslope == -np.inf or avgslope == np.inf or avgslope == 0):
				y1 = img.shape[0]
				y2 = int(y1*(3/5)) #only from the edge bottom up to 3/5th of the image
				x1 = int((y1-avgintercept)/avgslope)
				x2 = int((y2-avgintercept)/avgslope)
				#Draw left line on image
				cv2.line(img, (x1, y1), (x2, y2), color, thickness)
		if right_line:
			#Get average of all the right lines found
			right_avg_line = np.average(right_line,axis=0)
			#Calculate its coordinates from m & b
			avgslope,avgintercept = right_avg_line
			if not (avgslope == -np.inf or avgslope == np.inf or avgslope == 0):
				y1 = img.shape[0]
				y2 = int(y1*(3/5)) #only from the edge bottom up to 3/5th of the image
				x1 = int((y1-avgintercept)/avgslope)
				x2 = int((y2-avgintercept)/avgslope)
				cv2.line(img, (x1, y1), (x2, y2), color, thickness)




**Final Image with single lines to each side**
<figure>
 <img src="result_images/imageHoughLinesExtrapolated.png" width="380" alt="Hough Line Extrapolated" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Hough Line Extrapolated</p> 
 </figcaption>
</figure>
 <p></p> 
 

## Optional Challenge

I tried to make it work. I figured out that the video had another resolution (1280x720). Also I saw that the part of the car that you can see on the bottom edge was affecting the algorithm.
So I modified the region of interest to fit a bigger frame/image and to not include the bottom edge of the image.

<code>
#Since we have video images of 960x540 and 1280x720 we are applying differente
#shapes depending on the size of the image

if(imshape[1] == 960): 	#small image uses this mask
	vertices = [(150,imshape[0]),(480, 300), (490, 300), (imshape[1]-30,imshape[0])]
else:					#large image uses this mask
	vertices = [(int(imshape[1]*6/32),660),(620, 430), (710, 430), (imshape[1]-150,650)]
</code>

<p></p>

<figure>
    <div style="text-align: center;">
 <img src="result_images/imageAreaOfInteres Challenge1.png" style='margin-top:0;width: 380px; display: inline-block;' alt="Hough Lines masked Left Line"/>  <img src="result_images/imageHoughLinesExtrapolatedChallenge1.png" style='margin-top:0;width: 380px; display:inline-block;' alt="Hough Lines masked Right Line" />
        </div>
 <figcaption>
 <p><br></p> 
 <p style="text-align: center;clear: both; "> Challenge video uses a different mask shape</p> 
 </figcaption>
</figure>
 <p></p>

However, when the road is whiter, the algorithm does not work at all. I guess it needs something more robust
 <p></p> 
<figure>
 <img src="result_images/imageAreaOfInteresChallenge2.png" width="380" alt="Hough Line Extrapolated" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Whiter roads are not fun</p> 
 </figcaption>
</figure>
 <p></p> 
 